# Сравнение моделей

In [2]:
from catboost import CatBoostClassifier
import optuna
import pandas as pd
import wandb
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
!wandb login

wandb: Currently logged in as: buldakovn. Use `wandb login --relogin` to force relogin


In [4]:
models_path = [
    'catboost_baseline.model',
    'catboost_full.model',
    'catboost_with_text.model',
    'catboost_with_id.model',
    'catboost_with_text-class.model',
    'catboost_with_text_text-class.model',
    'catboost_with_text.model'
]

In [46]:
X_train = pd.read_csv('../data/unbalanced_data/train_data.csv')
Y_train = X_train['label']

X_valid = pd.read_csv('../data/unbalanced_data/valid_data.csv')
Y_valid = X_valid['label']

X_test = pd.read_csv('../data/unbalanced_data/test_data.csv')
Y_test = X_test['label']

In [6]:
%env WANDB_SILENT=True

env: WANDB_SILENT=True


In [65]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = CatBoostClassifier().load_model(full_path)
    params = model.get_params()
    threshold = 0.5
    model.set_probability_threshold(threshold)
    name = path.split('.')[0][9:]
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = model.predict(X_test[model.feature_names_])
    wandb.init(
        project="WB-reviews-classification",
        name = name,
        tags=[average, 'catboost'],
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
./catboost_full.model
./catboost_with_text.model


./catboost_with_id.model
./catboost_with_text-class.model


./catboost_with_text_text-class.model
./catboost_with_text.model


## Кастомный класс

In [66]:
class FakeReviewsClassifier:
    def __init__(self, model_path='./catboost_full.model', get_features=True):
        self.catboost_model: CatBoostClassifier = CatBoostClassifier().load_model(model_path)
        self.feature_names_ = self.catboost_model.feature_names_
        self.get_features = get_features
        
    def get_params(self):
        return self.catboost_model.get_params()

    def get_features(self, data: pd.Series):
        return get_features(data)

    def preproccess(self, data: pd.Series, get_features):
        data = data.drop(['f2', 'f4'], errors='ignore')
        if get_features:
            data = self.get_features(data)
            id_data = data.get(['id1_pred', 'id1_in_database', 'id2_pred', 'id2_in_database'])
            data = data.drop(['id1', 'id2', 'id3', 'text', 'label'])   
        else:
            id_data = data.get(['id1_pred', 'id1_in_database', 'id2_pred', 'id2_in_database'])
            data = data[self.feature_names_]
        return data, id_data
    
    # предсказание по id
    def id_predict(self, id_data: pd.Series, threshold):
        if not ('id1_in_database' in id_data.index):
            return None
        if id_data['id1_in_database']:
            return 1 if id_data['id1_pred'] >= threshold else 0
        return None
        
    def predict(self, data: pd.Series, 
                get_features=None, 
                id_predict = True,
                threshold = 0.5):
        if get_features is None:
            get_features = self.get_features
        data, id_data = self.preproccess(data, get_features)
        # если пользователь оставляет очень много фейков или наоборот – честных отзывов
        predict = None
        if id_predict and (not id_data is None):
            predict = self.id_predict(id_data, threshold=threshold)
        if predict is None:
            self.catboost_model.set_probability_threshold(threshold)
            predict = self.catboost_model.predict(data)
        return predict


In [67]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = FakeReviewsClassifier(full_path)
    params = model.get_params()
    name = path.split('.')[0][9:]
    threshold = 0.5
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)
    print(f1_score(Y_test, test_pred, average=average))
    wandb.init(
        # set the wandb project where this run will be logged
        project="WB-reviews-classification",
        name = name+'_custom',
        tags=[average, 'custom', 'catboost'],
        # track hyperparameters and run metadata
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
0.6552185005798484
./catboost_full.model
0.7591067304984511
./catboost_with_text.model
0.6244086361242759
./catboost_with_id.model
0.6389769816344933
./catboost_with_text-class.model
0.7525990515960083
./catboost_with_text_text-class.model
0.7745779672706752
./catboost_with_text.model
0.6244086361242759


In [70]:
for path in models_path:
    full_path = r"./"+path
    print(full_path)
    model = FakeReviewsClassifier(full_path)
    params = model.get_params()
    name = path.split('.')[0][9:]
    threshold = 0.55
    average='weighted'
    params['name'] = name
    params['threshold'] = threshold
    test_pred = X_test[model.feature_names_].apply(lambda x: model.predict(x, get_features=False, threshold=threshold), axis=1)
    print(f1_score(Y_test, test_pred, average=average))
    wandb.init(
        # set the wandb project where this run will be logged
        project="WB-reviews-classification",
        name = name+'_custom',
        tags=[average, 'custom', 'catboost'],
        # track hyperparameters and run metadata
        config=params, 
    )
    wandb.log({
        'test_f1': f1_score(Y_test, test_pred, average=average),
        'test_precision': precision_score(Y_test, test_pred, average=average),
        'test_recall': recall_score(Y_test, test_pred, average=average),
    })
    wandb.finish()

./catboost_baseline.model
0.6812413758548989
./catboost_full.model
0.7746631475755976
./catboost_with_text.model
0.6278301693860312
./catboost_with_id.model
0.6812178807406802
./catboost_with_text-class.model
0.7714258648711401
./catboost_with_text_text-class.model
0.785224236485383
./catboost_with_text.model
0.6278301693860312
